### R.F. - Predicting Electricity Use for UES-Seacoast Service Territory with 10 Years of ISO & UES Data
##### NO WEATHER DATA USED!!!

In [1]:
# Libraries used
import numpy as np
import pandas as pd
import time

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from pprint import pprint

### Predict UES load - 1 Day Ahead

In [2]:
# Read in data
data = pd.read_csv("ISO.aggregate - 1 day SHIFTED.csv")

# View first few rows of data
data.head()

,Month,Day,Year,TI,ISO.tot.dmd,LMP.DA.NH,LMP.RT.NH,UES.Cap,UES.Sea,UES.Sea.NEXT.DAY.LOAD,NH.RT.MWh,ME.RT.MWh,VT.RT.MWh,CT.RT.MWh,RI.RT.MWh,MA.SE.RT.MWh,MA.WC.RT.MWh,MA.NE.RT.MWh,Wk.Day
0,1,1,2010,1,13797.0,63.04,63.33,30.142,67.858,61.904,1168.0,1119.7,640.4,3206.8,825.4,1498.2,1774.3,2688.9,Fri
1,1,1,2010,2,13560.0,57.61,69.74,27.370,63.276,58.754,1091.8,1077.1,605.4,3031.6,781.9,1404.3,1676.8,2569.0,Fri
2,1,1,2010,3,13121.0,53.74,55.39,25.573,59.899,57.320,1046.4,1045.1,578.9,2902.9,749.7,1340.0,1621.8,2483.2,Fri
3,1,1,2010,4,13134.0,42.09,60.49,24.728,58.275,57.068,1021.5,1030.5,556.9,2827.7,729.7,1306.1,1588.5,2428.1,Fri
4,1,1,2010,5,13144.0,47.95,53.43,24.974,58.516,57.858,1018.0,1047.4,553.7,2820.7,730.0,1308.1,1586.7,2432.2,Fri


In [3]:
# Check dataframe data types
data.dtypes

Month                      int64
Day                        int64
Year                       int64
TI                         int64
ISO.tot.dmd              float64
LMP.DA.NH                float64
LMP.RT.NH                float64
UES.Cap                  float64
UES.Sea                  float64
UES.Sea.NEXT.DAY.LOAD    float64
NH.RT.MWh                float64
ME.RT.MWh                float64
VT.RT.MWh                float64
CT.RT.MWh                float64
RI.RT.MWh                float64
MA.SE.RT.MWh             float64
MA.WC.RT.MWh             float64
MA.NE.RT.MWh             float64
Wk.Day                    object
dtype: object

In [4]:
# Shape of dataframe
data.shape

(78864, 19)

In [5]:
# Descriptive statistics for each column
data.describe()

,Month,Day,Year,TI,ISO.tot.dmd,LMP.DA.NH,LMP.RT.NH,UES.Cap,UES.Sea,UES.Sea.NEXT.DAY.LOAD,NH.RT.MWh,ME.RT.MWh,VT.RT.MWh,CT.RT.MWh,RI.RT.MWh,MA.SE.RT.MWh,MA.WC.RT.MWh,MA.NE.RT.MWh
count,78864.000000,78864.000000,78864.000000,78864.00000,78864.000000,78864.000000,78864.000000,78864.000000,78864.000000,78864.000000,78864.000000,78864.000000,78864.000000,78864.000000,78864.000000,78864.000000,78864.000000,78864.000000
mean,6.522520,15.723182,2013.998783,12.50000,14580.600875,44.318309,43.562332,49.924784,76.881439,76.881482,1319.449053,1306.260874,644.164399,3482.634094,928.737496,1689.311137,1962.395218,2885.571864
std,3.447958,8.797753,2.581141,6.92223,2701.434331,34.645532,41.976925,16.058602,18.113627,18.113338,265.309210,205.406096,107.807006,771.813292,206.087265,389.960098,378.896406,570.384539
min,1.000000,1.000000,2010.000000,1.00000,8665.000000,-5.000000,-155.110000,5.395000,6.912000,6.912000,523.200000,681.800000,302.300000,1367.400000,371.800000,708.700000,736.900000,1729.700000
25%,4.000000,8.000000,2012.000000,6.75000,12636.022500,26.567500,24.197500,37.744000,64.281750,64.283500,1121.200000,1143.500000,558.900000,2937.975000,784.100000,1413.275000,1683.000000,2473.475000
50%,7.000000,16.000000,2014.000000,12.50000,14396.130000,36.180000,34.460000,49.499000,76.038000,76.038000,1333.100000,1329.600000,648.800000,3444.300000,916.600000,1659.400000,1951.600000,2869.400000
75%,10.000000,23.000000,2016.000000,18.25000,16121.000000,48.680000,47.760000,60.214000,86.302250,86.300250,1485.400000,1450.125000,721.500000,3915.500000,1032.300000,1887.400000,2199.025000,3210.900000
max,12.000000,31.000000,2018.000000,24.00000,27762.000000,785.000000,2493.150000,118.547000,168.430000,168.430000,2433.300000,2034.100000,1002.500000,7218.800000,4314.300000,3644.700000,3652.300000,5657.900000


In [6]:
# One-hot encoding
data = pd.get_dummies(data)

# Check dataframe data types
data.dtypes

Month                      int64
Day                        int64
Year                       int64
TI                         int64
ISO.tot.dmd              float64
LMP.DA.NH                float64
LMP.RT.NH                float64
UES.Cap                  float64
UES.Sea                  float64
UES.Sea.NEXT.DAY.LOAD    float64
NH.RT.MWh                float64
ME.RT.MWh                float64
VT.RT.MWh                float64
CT.RT.MWh                float64
RI.RT.MWh                float64
MA.SE.RT.MWh             float64
MA.WC.RT.MWh             float64
MA.NE.RT.MWh             float64
Wk.Day_Fri                 uint8
Wk.Day_Mon                 uint8
Wk.Day_Sat                 uint8
Wk.Day_Sun                 uint8
Wk.Day_Thu                 uint8
Wk.Day_Tue                 uint8
Wk.Day_Wed                 uint8
dtype: object

In [7]:
# Split data into target variable and predictor variables

# Target is the value we want to predict (in this case, MW of UES-Seacoast Load Region)
target = np.array(data['UES.Sea.NEXT.DAY.LOAD'])

# Remove the target from the predictors
# axis 1 refers to the columns
predictors = data.drop('UES.Sea.NEXT.DAY.LOAD', axis = 1)

# Saving predictor names for later use
predictor_list = list(predictors.columns)

# Convert to numpy array
predictors = np.array(predictors)

In [8]:
# Split the data into training and testing sets using sklearn
train_predictors, test_predictors, train_target, test_target = train_test_split(predictors, target, test_size = 0.25, random_state = 77)


In [9]:
# Check shape of train and test data
print('Training Predictors Shape:', train_predictors.shape)
print('Training Target Shape:', train_target.shape)
print('Testing Predictors Shape:', test_predictors.shape)
print('Testing Target Shape:', test_target.shape)

Training Predictors Shape: (59148, 24)
Training Target Shape: (59148,)
Testing Predictors Shape: (19716, 24)
Testing Target Shape: (19716,)


In [10]:
# Instantiate model with 300 decision trees
start_time = time.time()

rf = RandomForestRegressor(n_estimators = 300, random_state = 77)
# Train the model on training data
rf.fit(train_predictors, train_target)

end_time = time.time()

print("--- %s seconds ---" % (end_time - start_time))

--- 411.489000082016 seconds ---


In [11]:
# Use the forest's predict method on the test data
predictions = rf.predict(test_predictors)

# Calculate max and mean test_target value
test_target_max_1 = max(test_target)
test_target_mean_1 = np.mean(test_target)

# Calculate the absolute errors, calculate max and mean absolute error
errors = abs(predictions - test_target)
max_absolute_error_1 = max(errors)
mean_absolute_error_1 = np.mean(errors)

# Calculate mean absolute percentage error (MAPE), and Accuracy
mape_1 = 100 * (errors / test_target)
accuracy_1 = 100 - np.mean(mape_1)

# Print out some statistics, including accuracy
print('Target Variable Max Value:', round(test_target_max_1, 2), 'MW')
print('Max Absolute Error:', round(max_absolute_error_1, 2), 'MW')
print('Mean Target Variable Value:', round(test_target_mean_1, 2), 'MW')
print('Mean Absolute Error:', round(mean_absolute_error_1, 2), 'MW')
print('Accuracy:', round(accuracy_1, 2), '%.')

Target Variable Max Value: 166.65 MW
Max Absolute Error: 51.4 MW
Mean Target Variable Value: 77.05 MW
Mean Absolute Error: 3.5 MW
Accuracy: 95.58 %.


In [12]:
# Get numerical feature importances
importances = list(rf.feature_importances_)

# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(predictor_list, importances)]

# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)

# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

Variable: UES.Sea              Importance: 0.84
Variable: Month                Importance: 0.01
Variable: Day                  Importance: 0.01
Variable: TI                   Importance: 0.01
Variable: ISO.tot.dmd          Importance: 0.01
Variable: LMP.DA.NH            Importance: 0.01
Variable: LMP.RT.NH            Importance: 0.01
Variable: UES.Cap              Importance: 0.01
Variable: NH.RT.MWh            Importance: 0.01
Variable: ME.RT.MWh            Importance: 0.01
Variable: VT.RT.MWh            Importance: 0.01
Variable: CT.RT.MWh            Importance: 0.01
Variable: RI.RT.MWh            Importance: 0.01
Variable: MA.SE.RT.MWh         Importance: 0.01
Variable: MA.WC.RT.MWh         Importance: 0.01
Variable: MA.NE.RT.MWh         Importance: 0.01
Variable: Wk.Day_Sun           Importance: 0.01
Variable: Year                 Importance: 0.0
Variable: Wk.Day_Fri           Importance: 0.0
Variable: Wk.Day_Mon           Importance: 0.0
Variable: Wk.Day_Sat           Importance: 

In [13]:
# Look at parameters used by our current forest
print('Parameters currently in use:\n')
pprint(rf.get_params())

Parameters currently in use:

{'bootstrap': True,
 'criterion': 'mse',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 300,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 77,
 'verbose': 0,
 'warm_start': False}


In [204]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]

# Number of features to consider at every split
max_features = ['auto', 'sqrt']

# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)

# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]

# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]

# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint(random_grid)

{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}


In [ ]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()

# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=77, n_jobs = -1)

# Fit the random search model
rf_random.fit(train_predictors, train_target)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed: 109.8min


In [ ]:
# View the best parameters after fitting the random search
rf_random.best_params_

In [ ]:
# Create function for evaluating model performance
def evaluate(model, test_predictors, test_target):
    predictions = model.predict(test_predictors)
    errors = abs(predictions - test_target)
    mape = 100 * np.mean(errors / test_target)
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f} MW.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return accuracy

In [ ]:
# Compare base model with results of randomized search to see if there is an improvement
base_model = RandomForestRegressor(n_estimators = 300, random_state = 77)
base_model.fit(train_predictors, train_target)
base_accuracy = evaluate(base_model, test_predictors, test_target)


best_random = rf_random.best_estimator_
random_accuracy = evaluate(best_random, test_predictors, test_target)

print('Improvement of {:0.2f}%.'.format( 100 * (random_accuracy - base_accuracy) / base_accuracy))

In [ ]:
### See if performance can be improved further by using GridSearchCV

# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True, False],
    'max_depth': [70, 80, 90, 100],
    'max_features': [2, 3, 5],
    'min_samples_leaf': [1, 2, 4],
    'min_samples_split': [1, 2, 4],
    'n_estimators': [100, 300, 600, 1000]
}
# Create a base model
rf = RandomForestRegressor()

# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)

In [ ]:
# Fit the grid search to the data and view the best parameters
grid_search.fit(train_predictors, train_target)
grid_search.best_params_

In [ ]:
# Use best estimator from GridSearchCV
best_grid = grid_search.best_estimator_
grid_accuracy = evaluate(best_grid, test_predictors, test_target)

# Re-run base grid case to be able to compare accuracies
base_model = RandomForestRegressor(n_estimators = 300, random_state = 77)
base_model.fit(train_predictors, train_target)
base_accuracy = evaluate(base_model, test_predictors, test_target)

print('Improvement of {:0.2f}%.'.format( 100 * (grid_accuracy - base_accuracy) / base_accuracy))

### Predict UES load - 2 Days Ahead

In [14]:
# Read in data
data = pd.read_csv("ISO.aggregate - 2 day SHIFTED.csv")

# View first few rows of data
data.head()

,Month,Day,Year,TI,ISO.tot.dmd,LMP.DA.NH,LMP.RT.NH,UES.Cap,UES.Sea,UES.Sea.2.DAYS.AHEAD,NH.RT.MWh,ME.RT.MWh,VT.RT.MWh,CT.RT.MWh,RI.RT.MWh,MA.SE.RT.MWh,MA.WC.RT.MWh,MA.NE.RT.MWh,Wk.Day
0,1,1,2010,1,13797.0,63.04,63.33,30.142,67.858,67.512,1168.0,1119.7,640.4,3206.8,825.4,1498.2,1774.3,2688.9,Fri
1,1,1,2010,2,13560.0,57.61,69.74,27.370,63.276,64.690,1091.8,1077.1,605.4,3031.6,781.9,1404.3,1676.8,2569.0,Fri
2,1,1,2010,3,13121.0,53.74,55.39,25.573,59.899,63.511,1046.4,1045.1,578.9,2902.9,749.7,1340.0,1621.8,2483.2,Fri
3,1,1,2010,4,13134.0,42.09,60.49,24.728,58.275,63.021,1021.5,1030.5,556.9,2827.7,729.7,1306.1,1588.5,2428.1,Fri
4,1,1,2010,5,13144.0,47.95,53.43,24.974,58.516,63.480,1018.0,1047.4,553.7,2820.7,730.0,1308.1,1586.7,2432.2,Fri


In [15]:
# Shape of dataframe
data.shape

(78840, 19)

In [16]:
# One-hot encoding
data = pd.get_dummies(data)

# Check dataframe data types
data.dtypes

Month                     int64
Day                       int64
Year                      int64
TI                        int64
ISO.tot.dmd             float64
LMP.DA.NH               float64
LMP.RT.NH               float64
UES.Cap                 float64
UES.Sea                 float64
UES.Sea.2.DAYS.AHEAD    float64
NH.RT.MWh               float64
ME.RT.MWh               float64
VT.RT.MWh               float64
CT.RT.MWh               float64
RI.RT.MWh               float64
MA.SE.RT.MWh            float64
MA.WC.RT.MWh            float64
MA.NE.RT.MWh            float64
Wk.Day_Fri                uint8
Wk.Day_Mon                uint8
Wk.Day_Sat                uint8
Wk.Day_Sun                uint8
Wk.Day_Thu                uint8
Wk.Day_Tue                uint8
Wk.Day_Wed                uint8
dtype: object

In [17]:
# Split data into target variable and predictor variables

# Target is the value we want to predict (in this case, MW of UES-Seacoast Load Region)
target = np.array(data['UES.Sea.2.DAYS.AHEAD'])

# Remove the target from the predictors
# axis 1 refers to the columns
predictors = data.drop('UES.Sea.2.DAYS.AHEAD', axis = 1)

# Saving predictor names for later use
predictor_list = list(predictors.columns)

# Convert to numpy array
predictors = np.array(predictors)

In [18]:
# Split the data into training and testing sets using sklearn
train_predictors, test_predictors, train_target, test_target = train_test_split(predictors, target, test_size = 0.25, random_state = 77)


In [19]:
# Check shape of train and test data
print('Training Predictors Shape:', train_predictors.shape)
print('Training Target Shape:', train_target.shape)
print('Testing Predictors Shape:', test_predictors.shape)
print('Testing Target Shape:', test_target.shape)

Training Predictors Shape: (59130, 24)
Training Target Shape: (59130,)
Testing Predictors Shape: (19710, 24)
Testing Target Shape: (19710,)


In [20]:
# Instantiate model with 300 decision trees
start_time = time.time()

rf = RandomForestRegressor(n_estimators = 300, random_state = 77)
# Train the model on training data
rf.fit(train_predictors, train_target)

end_time = time.time()

print("--- %s seconds ---" % (end_time - start_time))

--- 386.6080000400543 seconds ---


In [21]:
# Use the forest's predict method on the test data
predictions = rf.predict(test_predictors)

# Calculate max and mean test_target value
test_target_max_2 = max(test_target)
test_target_mean_2 = np.mean(test_target)

# Calculate the absolute errors, calculate max and mean absolute error
errors = abs(predictions - test_target)
max_absolute_error_2 = max(errors)
mean_absolute_error_2 = np.mean(errors)

# Calculate mean absolute percentage error (MAPE), and Accuracy
mape_2 = 100 * (errors / test_target)
accuracy_2 = 100 - np.mean(mape_2)

# Print out some statistics, including accuracy
print('Target Variable Max Value:', round(test_target_max_2, 2), 'MW')
print('Max Absolute Error:', round(max_absolute_error_2, 2), 'MW')
print('Mean Target Variable Value:', round(test_target_mean_2, 2), 'MW')
print('Mean Absolute Error:', round(mean_absolute_error_2, 2), 'MW')
print('Accuracy:', round(accuracy_2, 2), '%.')

Target Variable Max Value: 168.43 MW
Max Absolute Error: 53.69 MW
Mean Target Variable Value: 76.92 MW
Mean Absolute Error: 4.21 MW
Accuracy: 94.64 %.


In [22]:
# Get numerical feature importances
importances = list(rf.feature_importances_)

# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(predictor_list, importances)]

# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)

# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

Variable: UES.Sea              Importance: 0.7
Variable: TI                   Importance: 0.05
Variable: Month                Importance: 0.02
Variable: Day                  Importance: 0.02
Variable: LMP.DA.NH            Importance: 0.02
Variable: LMP.RT.NH            Importance: 0.02
Variable: ME.RT.MWh            Importance: 0.02
Variable: VT.RT.MWh            Importance: 0.02
Variable: MA.SE.RT.MWh         Importance: 0.02
Variable: Year                 Importance: 0.01
Variable: ISO.tot.dmd          Importance: 0.01
Variable: UES.Cap              Importance: 0.01
Variable: NH.RT.MWh            Importance: 0.01
Variable: CT.RT.MWh            Importance: 0.01
Variable: RI.RT.MWh            Importance: 0.01
Variable: MA.WC.RT.MWh         Importance: 0.01
Variable: MA.NE.RT.MWh         Importance: 0.01
Variable: Wk.Day_Fri           Importance: 0.01
Variable: Wk.Day_Sat           Importance: 0.01
Variable: Wk.Day_Sun           Importance: 0.01
Variable: Wk.Day_Thu           Importance

### Predict UES load - 3 Days Ahead

In [23]:
# Read in data
data = pd.read_csv("ISO.aggregate - 3 day SHIFTED.csv")

# View first few rows of data
data.head()

,Month,Day,Year,TI,ISO.tot.dmd,LMP.DA.NH,LMP.RT.NH,UES.Cap,UES.Sea,UES.Sea.3.DAYS.AHEAD,NH.RT.MWh,ME.RT.MWh,VT.RT.MWh,CT.RT.MWh,RI.RT.MWh,MA.SE.RT.MWh,MA.WC.RT.MWh,MA.NE.RT.MWh,Wk.Day
0,1,1,2010,1,13797.0,63.04,63.33,30.142,67.858,61.488,1168.0,1119.7,640.4,3206.8,825.4,1498.2,1774.3,2688.9,Fri
1,1,1,2010,2,13560.0,57.61,69.74,27.370,63.276,59.038,1091.8,1077.1,605.4,3031.6,781.9,1404.3,1676.8,2569.0,Fri
2,1,1,2010,3,13121.0,53.74,55.39,25.573,59.899,57.848,1046.4,1045.1,578.9,2902.9,749.7,1340.0,1621.8,2483.2,Fri
3,1,1,2010,4,13134.0,42.09,60.49,24.728,58.275,58.058,1021.5,1030.5,556.9,2827.7,729.7,1306.1,1588.5,2428.1,Fri
4,1,1,2010,5,13144.0,47.95,53.43,24.974,58.516,61.047,1018.0,1047.4,553.7,2820.7,730.0,1308.1,1586.7,2432.2,Fri


In [24]:
# Shape of dataframe
data.shape

(78816, 19)

In [25]:
# One-hot encoding
data = pd.get_dummies(data)

# Check dataframe data types
data.dtypes

Month                     int64
Day                       int64
Year                      int64
TI                        int64
ISO.tot.dmd             float64
LMP.DA.NH               float64
LMP.RT.NH               float64
UES.Cap                 float64
UES.Sea                 float64
UES.Sea.3.DAYS.AHEAD    float64
NH.RT.MWh               float64
ME.RT.MWh               float64
VT.RT.MWh               float64
CT.RT.MWh               float64
RI.RT.MWh               float64
MA.SE.RT.MWh            float64
MA.WC.RT.MWh            float64
MA.NE.RT.MWh            float64
Wk.Day_Fri                uint8
Wk.Day_Mon                uint8
Wk.Day_Sat                uint8
Wk.Day_Sun                uint8
Wk.Day_Thu                uint8
Wk.Day_Tue                uint8
Wk.Day_Wed                uint8
dtype: object

In [26]:
# Split data into target variable and predictor variables

# Target is the value we want to predict (in this case, MW of UES-Seacoast Load Region)
target = np.array(data['UES.Sea.3.DAYS.AHEAD'])

# Remove the target from the predictors
# axis 1 refers to the columns
predictors = data.drop('UES.Sea.3.DAYS.AHEAD', axis = 1)

# Saving predictor names for later use
predictor_list = list(predictors.columns)

# Convert to numpy array
predictors = np.array(predictors)

In [27]:
# Split the data into training and testing sets using sklearn
train_predictors, test_predictors, train_target, test_target = train_test_split(predictors, target, test_size = 0.25, random_state = 77)


In [28]:
# Check shape of train and test data
print('Training Predictors Shape:', train_predictors.shape)
print('Training Target Shape:', train_target.shape)
print('Testing Predictors Shape:', test_predictors.shape)
print('Testing Target Shape:', test_target.shape)

Training Predictors Shape: (59112, 24)
Training Target Shape: (59112,)
Testing Predictors Shape: (19704, 24)
Testing Target Shape: (19704,)


In [29]:
# Instantiate model with 300 decision trees
start_time = time.time()

rf = RandomForestRegressor(n_estimators = 300, random_state = 77)
# Train the model on training data
rf.fit(train_predictors, train_target)

end_time = time.time()

print("--- %s seconds ---" % (end_time - start_time))

--- 375.510999917984 seconds ---


In [30]:
# Use the forest's predict method on the test data
predictions = rf.predict(test_predictors)

# Calculate max and mean test_target value
test_target_max_3 = max(test_target)
test_target_mean_3 = np.mean(test_target)

# Calculate the absolute errors, calculate max and mean absolute error
errors = abs(predictions - test_target)
max_absolute_error_3 = max(errors)
mean_absolute_error_3 = np.mean(errors)

# Calculate mean absolute percentage error (MAPE), and Accuracy
mape_3 = 100 * (errors / test_target)
accuracy_3 = 100 - np.mean(mape_3)

# Print out some statistics, including accuracy
print('Target Variable Max Value:', round(test_target_max_3, 2), 'MW')
print('Max Absolute Error:', round(max_absolute_error_3, 2), 'MW')
print('Mean Target Variable Value:', round(test_target_mean_3, 2), 'MW')
print('Mean Absolute Error:', round(mean_absolute_error_3, 2), 'MW')
print('Accuracy:', round(accuracy_3, 2), '%.')

Target Variable Max Value: 168.42 MW
Max Absolute Error: 54.49 MW
Mean Target Variable Value: 77.07 MW
Mean Absolute Error: 4.12 MW
Accuracy: 94.66 %.


In [31]:
# Get numerical feature importances
importances = list(rf.feature_importances_)

# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(predictor_list, importances)]

# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)

# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

Variable: UES.Sea              Importance: 0.54
Variable: TI                   Importance: 0.14
Variable: Month                Importance: 0.04
Variable: MA.SE.RT.MWh         Importance: 0.04
Variable: Day                  Importance: 0.03
Variable: LMP.DA.NH            Importance: 0.02
Variable: LMP.RT.NH            Importance: 0.02
Variable: ME.RT.MWh            Importance: 0.02
Variable: VT.RT.MWh            Importance: 0.02
Variable: Year                 Importance: 0.01
Variable: ISO.tot.dmd          Importance: 0.01
Variable: UES.Cap              Importance: 0.01
Variable: NH.RT.MWh            Importance: 0.01
Variable: CT.RT.MWh            Importance: 0.01
Variable: RI.RT.MWh            Importance: 0.01
Variable: MA.WC.RT.MWh         Importance: 0.01
Variable: MA.NE.RT.MWh         Importance: 0.01
Variable: Wk.Day_Sat           Importance: 0.01
Variable: Wk.Day_Thu           Importance: 0.01
Variable: Wk.Day_Wed           Importance: 0.01
Variable: Wk.Day_Fri           Importanc

### Predict UES load - 4 Days Ahead

In [32]:
# Read in data
data = pd.read_csv("ISO.aggregate - 4 day SHIFTED.csv")

# View first few rows of data
data.head()

,Month,Day,Year,TI,ISO.tot.dmd,LMP.DA.NH,LMP.RT.NH,UES.Cap,UES.Sea,UES.Sea.4.DAYS.AHEAD,NH.RT.MWh,ME.RT.MWh,VT.RT.MWh,CT.RT.MWh,RI.RT.MWh,MA.SE.RT.MWh,MA.WC.RT.MWh,MA.NE.RT.MWh,Wk.Day
0,1,1,2010,1,13797.0,63.04,63.33,30.142,67.858,61.940,1168.0,1119.7,640.4,3206.8,825.4,1498.2,1774.3,2688.9,Fri
1,1,1,2010,2,13560.0,57.61,69.74,27.370,63.276,59.924,1091.8,1077.1,605.4,3031.6,781.9,1404.3,1676.8,2569.0,Fri
2,1,1,2010,3,13121.0,53.74,55.39,25.573,59.899,58.818,1046.4,1045.1,578.9,2902.9,749.7,1340.0,1621.8,2483.2,Fri
3,1,1,2010,4,13134.0,42.09,60.49,24.728,58.275,58.968,1021.5,1030.5,556.9,2827.7,729.7,1306.1,1588.5,2428.1,Fri
4,1,1,2010,5,13144.0,47.95,53.43,24.974,58.516,61.194,1018.0,1047.4,553.7,2820.7,730.0,1308.1,1586.7,2432.2,Fri


In [33]:
# Shape of dataframe
data.shape

(78792, 19)

In [34]:
# One-hot encoding
data = pd.get_dummies(data)

# Check dataframe data types
data.dtypes

Month                     int64
Day                       int64
Year                      int64
TI                        int64
ISO.tot.dmd             float64
LMP.DA.NH               float64
LMP.RT.NH               float64
UES.Cap                 float64
UES.Sea                 float64
UES.Sea.4.DAYS.AHEAD    float64
NH.RT.MWh               float64
ME.RT.MWh               float64
VT.RT.MWh               float64
CT.RT.MWh               float64
RI.RT.MWh               float64
MA.SE.RT.MWh            float64
MA.WC.RT.MWh            float64
MA.NE.RT.MWh            float64
Wk.Day_Fri                uint8
Wk.Day_Mon                uint8
Wk.Day_Sat                uint8
Wk.Day_Sun                uint8
Wk.Day_Thu                uint8
Wk.Day_Tue                uint8
Wk.Day_Wed                uint8
dtype: object

In [35]:
# Split data into target variable and predictor variables

# Target is the value we want to predict (in this case, MW of UES-Seacoast Load Region)
target = np.array(data['UES.Sea.4.DAYS.AHEAD'])

# Remove the target from the predictors
# axis 1 refers to the columns
predictors = data.drop('UES.Sea.4.DAYS.AHEAD', axis = 1)

# Saving predictor names for later use
predictor_list = list(predictors.columns)

# Convert to numpy array
predictors = np.array(predictors)

In [36]:
# Split the data into training and testing sets using sklearn
train_predictors, test_predictors, train_target, test_target = train_test_split(predictors, target, test_size = 0.25, random_state = 77)


In [37]:
# Check shape of train and test data
print('Training Predictors Shape:', train_predictors.shape)
print('Training Target Shape:', train_target.shape)
print('Testing Predictors Shape:', test_predictors.shape)
print('Testing Target Shape:', test_target.shape)

Training Predictors Shape: (59094, 24)
Training Target Shape: (59094,)
Testing Predictors Shape: (19698, 24)
Testing Target Shape: (19698,)


In [38]:
# Instantiate model with 300 decision trees
start_time = time.time()

rf = RandomForestRegressor(n_estimators = 300, random_state = 77)
# Train the model on training data
rf.fit(train_predictors, train_target)

end_time = time.time()

print("--- %s seconds ---" % (end_time - start_time))

--- 366.3789999485016 seconds ---


In [39]:
# Use the forest's predict method on the test data
predictions = rf.predict(test_predictors)

# Calculate max and mean test_target value
test_target_max_4 = max(test_target)
test_target_mean_4 = np.mean(test_target)

# Calculate the absolute errors, calculate max and mean absolute error
errors = abs(predictions - test_target)
max_absolute_error_4 = max(errors)
mean_absolute_error_4 = np.mean(errors)

# Calculate mean absolute percentage error (MAPE), and Accuracy
mape_4 = 100 * (errors / test_target)
accuracy_4 = 100 - np.mean(mape_4)

# Print out some statistics, including accuracy
print('Target Variable Max Value:', round(test_target_max_4, 2), 'MW')
print('Max Absolute Error:', round(max_absolute_error_4, 2), 'MW')
print('Mean Target Variable Value:', round(test_target_mean_4, 2), 'MW')
print('Mean Absolute Error:', round(mean_absolute_error_4, 2), 'MW')
print('Accuracy:', round(accuracy_4, 2), '%.')

Target Variable Max Value: 168.43 MW
Max Absolute Error: 54.32 MW
Mean Target Variable Value: 77.13 MW
Mean Absolute Error: 3.97 MW
Accuracy: 94.86 %.


In [40]:
# Get numerical feature importances
importances = list(rf.feature_importances_)

# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(predictor_list, importances)]

# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)

# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

Variable: UES.Sea              Importance: 0.52
Variable: TI                   Importance: 0.12
Variable: MA.SE.RT.MWh         Importance: 0.06
Variable: Month                Importance: 0.05
Variable: Day                  Importance: 0.03
Variable: Year                 Importance: 0.02
Variable: LMP.DA.NH            Importance: 0.02
Variable: LMP.RT.NH            Importance: 0.02
Variable: ME.RT.MWh            Importance: 0.02
Variable: VT.RT.MWh            Importance: 0.02
Variable: MA.NE.RT.MWh         Importance: 0.02
Variable: ISO.tot.dmd          Importance: 0.01
Variable: UES.Cap              Importance: 0.01
Variable: NH.RT.MWh            Importance: 0.01
Variable: CT.RT.MWh            Importance: 0.01
Variable: RI.RT.MWh            Importance: 0.01
Variable: MA.WC.RT.MWh         Importance: 0.01
Variable: Wk.Day_Sat           Importance: 0.01
Variable: Wk.Day_Tue           Importance: 0.01
Variable: Wk.Day_Wed           Importance: 0.01
Variable: Wk.Day_Fri           Importanc

### Predict UES load - 5 Days Ahead

In [41]:
# Read in data
data = pd.read_csv("ISO.aggregate - 5 day SHIFTED.csv")

# View first few rows of data
data.head()

,Month,Day,Year,TI,ISO.tot.dmd,LMP.DA.NH,LMP.RT.NH,UES.Cap,UES.Sea,UES.Sea.5.DAYS.AHEAD,NH.RT.MWh,ME.RT.MWh,VT.RT.MWh,CT.RT.MWh,RI.RT.MWh,MA.SE.RT.MWh,MA.WC.RT.MWh,MA.NE.RT.MWh,Wk.Day
0,1,1,2010,1,13797.0,63.04,63.33,30.142,67.858,61.512,1168.0,1119.7,640.4,3206.8,825.4,1498.2,1774.3,2688.9,Fri
1,1,1,2010,2,13560.0,57.61,69.74,27.370,63.276,59.122,1091.8,1077.1,605.4,3031.6,781.9,1404.3,1676.8,2569.0,Fri
2,1,1,2010,3,13121.0,53.74,55.39,25.573,59.899,58.310,1046.4,1045.1,578.9,2902.9,749.7,1340.0,1621.8,2483.2,Fri
3,1,1,2010,4,13134.0,42.09,60.49,24.728,58.275,58.611,1021.5,1030.5,556.9,2827.7,729.7,1306.1,1588.5,2428.1,Fri
4,1,1,2010,5,13144.0,47.95,53.43,24.974,58.516,60.988,1018.0,1047.4,553.7,2820.7,730.0,1308.1,1586.7,2432.2,Fri


In [42]:
# Shape of dataframe
data.shape

(78768, 19)

In [43]:
# One-hot encoding
data = pd.get_dummies(data)

# Check dataframe data types
data.dtypes

Month                     int64
Day                       int64
Year                      int64
TI                        int64
ISO.tot.dmd             float64
LMP.DA.NH               float64
LMP.RT.NH               float64
UES.Cap                 float64
UES.Sea                 float64
UES.Sea.5.DAYS.AHEAD    float64
NH.RT.MWh               float64
ME.RT.MWh               float64
VT.RT.MWh               float64
CT.RT.MWh               float64
RI.RT.MWh               float64
MA.SE.RT.MWh            float64
MA.WC.RT.MWh            float64
MA.NE.RT.MWh            float64
Wk.Day_Fri                uint8
Wk.Day_Mon                uint8
Wk.Day_Sat                uint8
Wk.Day_Sun                uint8
Wk.Day_Thu                uint8
Wk.Day_Tue                uint8
Wk.Day_Wed                uint8
dtype: object

In [44]:
# Split data into target variable and predictor variables

# Target is the value we want to predict (in this case, MW of UES-Seacoast Load Region)
target = np.array(data['UES.Sea.5.DAYS.AHEAD'])

# Remove the target from the predictors
# axis 1 refers to the columns
predictors = data.drop('UES.Sea.5.DAYS.AHEAD', axis = 1)

# Saving predictor names for later use
predictor_list = list(predictors.columns)

# Convert to numpy array
predictors = np.array(predictors)

In [45]:
# Split the data into training and testing sets using sklearn
train_predictors, test_predictors, train_target, test_target = train_test_split(predictors, target, test_size = 0.25, random_state = 77)


In [46]:
# Check shape of train and test data
print('Training Predictors Shape:', train_predictors.shape)
print('Training Target Shape:', train_target.shape)
print('Testing Predictors Shape:', test_predictors.shape)
print('Testing Target Shape:', test_target.shape)

Training Predictors Shape: (59076, 24)
Training Target Shape: (59076,)
Testing Predictors Shape: (19692, 24)
Testing Target Shape: (19692,)


In [47]:
# Instantiate model with 300 decision trees
start_time = time.time()

rf = RandomForestRegressor(n_estimators = 300, random_state = 77)
# Train the model on training data
rf.fit(train_predictors, train_target)

end_time = time.time()

print("--- %s seconds ---" % (end_time - start_time))

--- 367.6640000343323 seconds ---


In [48]:
# Use the forest's predict method on the test data
predictions = rf.predict(test_predictors)

# Calculate max and mean test_target value
test_target_max_5 = max(test_target)
test_target_mean_5 = np.mean(test_target)

# Calculate the absolute errors, calculate max and mean absolute error
errors = abs(predictions - test_target)
max_absolute_error_5 = max(errors)
mean_absolute_error_5 = np.mean(errors)

# Calculate mean absolute percentage error (MAPE), and Accuracy
mape_5 = 100 * (errors / test_target)
accuracy_5 = 100 - np.mean(mape_5)

# Print out some statistics, including accuracy
print('Target Variable Max Value:', round(test_target_max_5, 2), 'MW')
print('Max Absolute Error:', round(max_absolute_error_5, 2), 'MW')
print('Mean Target Variable Value:', round(test_target_mean_5, 2), 'MW')
print('Mean Absolute Error:', round(mean_absolute_error_5, 2), 'MW')
print('Accuracy:', round(accuracy_5, 2), '%.')

Target Variable Max Value: 165.69 MW
Max Absolute Error: 59.83 MW
Mean Target Variable Value: 77.08 MW
Mean Absolute Error: 3.93 MW
Accuracy: 94.89 %.


In [49]:
# Get numerical feature importances
importances = list(rf.feature_importances_)

# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(predictor_list, importances)]

# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)

# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

Variable: UES.Sea              Importance: 0.5
Variable: TI                   Importance: 0.13
Variable: MA.SE.RT.MWh         Importance: 0.07
Variable: Month                Importance: 0.06
Variable: Day                  Importance: 0.04
Variable: Year                 Importance: 0.02
Variable: LMP.DA.NH            Importance: 0.02
Variable: ME.RT.MWh            Importance: 0.02
Variable: VT.RT.MWh            Importance: 0.02
Variable: ISO.tot.dmd          Importance: 0.01
Variable: LMP.RT.NH            Importance: 0.01
Variable: UES.Cap              Importance: 0.01
Variable: NH.RT.MWh            Importance: 0.01
Variable: CT.RT.MWh            Importance: 0.01
Variable: RI.RT.MWh            Importance: 0.01
Variable: MA.WC.RT.MWh         Importance: 0.01
Variable: MA.NE.RT.MWh         Importance: 0.01
Variable: Wk.Day_Mon           Importance: 0.01
Variable: Wk.Day_Tue           Importance: 0.01
Variable: Wk.Day_Fri           Importance: 0.0
Variable: Wk.Day_Sat           Importance:

### Predict UES load - 6 Days Ahead

In [50]:
# Read in data
data = pd.read_csv("ISO.aggregate - 6 day SHIFTED.csv")

# View first few rows of data
data.head()

,Month,Day,Year,TI,ISO.tot.dmd,LMP.DA.NH,LMP.RT.NH,UES.Cap,UES.Sea,UES.Sea.6.DAYS.AHEAD,NH.RT.MWh,ME.RT.MWh,VT.RT.MWh,CT.RT.MWh,RI.RT.MWh,MA.SE.RT.MWh,MA.WC.RT.MWh,MA.NE.RT.MWh,Wk.Day
0,1,1,2010,1,13797.0,63.04,63.33,30.142,67.858,62.741,1168.0,1119.7,640.4,3206.8,825.4,1498.2,1774.3,2688.9,Fri
1,1,1,2010,2,13560.0,57.61,69.74,27.370,63.276,60.452,1091.8,1077.1,605.4,3031.6,781.9,1404.3,1676.8,2569.0,Fri
2,1,1,2010,3,13121.0,53.74,55.39,25.573,59.899,59.588,1046.4,1045.1,578.9,2902.9,749.7,1340.0,1621.8,2483.2,Fri
3,1,1,2010,4,13134.0,42.09,60.49,24.728,58.275,59.829,1021.5,1030.5,556.9,2827.7,729.7,1306.1,1588.5,2428.1,Fri
4,1,1,2010,5,13144.0,47.95,53.43,24.974,58.516,62.058,1018.0,1047.4,553.7,2820.7,730.0,1308.1,1586.7,2432.2,Fri


In [51]:
# Shape of dataframe
data.shape

(78744, 19)

In [52]:
# One-hot encoding
data = pd.get_dummies(data)

# Check dataframe data types
data.dtypes

Month                     int64
Day                       int64
Year                      int64
TI                        int64
ISO.tot.dmd             float64
LMP.DA.NH               float64
LMP.RT.NH               float64
UES.Cap                 float64
UES.Sea                 float64
UES.Sea.6.DAYS.AHEAD    float64
NH.RT.MWh               float64
ME.RT.MWh               float64
VT.RT.MWh               float64
CT.RT.MWh               float64
RI.RT.MWh               float64
MA.SE.RT.MWh            float64
MA.WC.RT.MWh            float64
MA.NE.RT.MWh            float64
Wk.Day_Fri                uint8
Wk.Day_Mon                uint8
Wk.Day_Sat                uint8
Wk.Day_Sun                uint8
Wk.Day_Thu                uint8
Wk.Day_Tue                uint8
Wk.Day_Wed                uint8
dtype: object

In [53]:
# Split data into target variable and predictor variables

# Target is the value we want to predict (in this case, MW of UES-Seacoast Load Region)
target = np.array(data['UES.Sea.6.DAYS.AHEAD'])

# Remove the target from the predictors
# axis 1 refers to the columns
predictors = data.drop('UES.Sea.6.DAYS.AHEAD', axis = 1)

# Saving predictor names for later use
predictor_list = list(predictors.columns)

# Convert to numpy array
predictors = np.array(predictors)

In [54]:
# Split the data into training and testing sets using sklearn
train_predictors, test_predictors, train_target, test_target = train_test_split(predictors, target, test_size = 0.25, random_state = 77)


In [55]:
# Check shape of train and test data
print('Training Predictors Shape:', train_predictors.shape)
print('Training Target Shape:', train_target.shape)
print('Testing Predictors Shape:', test_predictors.shape)
print('Testing Target Shape:', test_target.shape)

Training Predictors Shape: (59058, 24)
Training Target Shape: (59058,)
Testing Predictors Shape: (19686, 24)
Testing Target Shape: (19686,)


In [56]:
# Instantiate model with 300 decision trees
start_time = time.time()

rf = RandomForestRegressor(n_estimators = 300, random_state = 77)
# Train the model on training data
rf.fit(train_predictors, train_target)

end_time = time.time()

print("--- %s seconds ---" % (end_time - start_time))

--- 368.3139998912811 seconds ---


In [57]:
# Use the forest's predict method on the test data
predictions = rf.predict(test_predictors)

# Calculate max and mean test_target value
test_target_max_6 = max(test_target)
test_target_mean_6 = np.mean(test_target)

# Calculate the absolute errors, calculate max and mean absolute error
errors = abs(predictions - test_target)
max_absolute_error_6 = max(errors)
mean_absolute_error_6 = np.mean(errors)

# Calculate mean absolute percentage error (MAPE), and Accuracy
mape_6 = 100 * (errors / test_target)
accuracy_6 = 100 - np.mean(mape_6)

# Print out some statistics, including accuracy
print('Target Variable Max Value:', round(test_target_max_6, 2), 'MW')
print('Max Absolute Error:', round(max_absolute_error_6, 2), 'MW')
print('Mean Target Variable Value:', round(test_target_mean_6, 2), 'MW')
print('Mean Absolute Error:', round(mean_absolute_error_6, 2), 'MW')
print('Accuracy:', round(accuracy_6, 2), '%.')

Target Variable Max Value: 166.73 MW
Max Absolute Error: 56.44 MW
Mean Target Variable Value: 76.96 MW
Mean Absolute Error: 4.03 MW
Accuracy: 94.73 %.


In [58]:
# Get numerical feature importances
importances = list(rf.feature_importances_)

# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(predictor_list, importances)]

# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)

# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

Variable: UES.Sea              Importance: 0.6
Variable: MA.SE.RT.MWh         Importance: 0.07
Variable: Month                Importance: 0.05
Variable: TI                   Importance: 0.05
Variable: Day                  Importance: 0.03
Variable: Year                 Importance: 0.02
Variable: LMP.DA.NH            Importance: 0.02
Variable: LMP.RT.NH            Importance: 0.02
Variable: ME.RT.MWh            Importance: 0.02
Variable: VT.RT.MWh            Importance: 0.02
Variable: ISO.tot.dmd          Importance: 0.01
Variable: UES.Cap              Importance: 0.01
Variable: NH.RT.MWh            Importance: 0.01
Variable: CT.RT.MWh            Importance: 0.01
Variable: RI.RT.MWh            Importance: 0.01
Variable: MA.WC.RT.MWh         Importance: 0.01
Variable: MA.NE.RT.MWh         Importance: 0.01
Variable: Wk.Day_Mon           Importance: 0.01
Variable: Wk.Day_Fri           Importance: 0.0
Variable: Wk.Day_Sat           Importance: 0.0
Variable: Wk.Day_Sun           Importance: 

### Predict UES load - 7 Days Ahead

In [59]:
# Read in data
data = pd.read_csv("ISO.aggregate - 7 day SHIFTED.csv")

# View first few rows of data
data.head()

,Month,Day,Year,TI,ISO.tot.dmd,LMP.DA.NH,LMP.RT.NH,UES.Cap,UES.Sea,UES.Sea.7.DAYS.AHEAD,NH.RT.MWh,ME.RT.MWh,VT.RT.MWh,CT.RT.MWh,RI.RT.MWh,MA.SE.RT.MWh,MA.WC.RT.MWh,MA.NE.RT.MWh,Wk.Day
0,1,1,2010,1,13797.0,63.04,63.33,30.142,67.858,62.538,1168.0,1119.7,640.4,3206.8,825.4,1498.2,1774.3,2688.9,Fri
1,1,1,2010,2,13560.0,57.61,69.74,27.370,63.276,60.144,1091.8,1077.1,605.4,3031.6,781.9,1404.3,1676.8,2569.0,Fri
2,1,1,2010,3,13121.0,53.74,55.39,25.573,59.899,58.989,1046.4,1045.1,578.9,2902.9,749.7,1340.0,1621.8,2483.2,Fri
3,1,1,2010,4,13134.0,42.09,60.49,24.728,58.275,59.150,1021.5,1030.5,556.9,2827.7,729.7,1306.1,1588.5,2428.1,Fri
4,1,1,2010,5,13144.0,47.95,53.43,24.974,58.516,61.425,1018.0,1047.4,553.7,2820.7,730.0,1308.1,1586.7,2432.2,Fri


In [60]:
# Shape of dataframe
data.shape

(78720, 19)

In [61]:
# One-hot encoding
data = pd.get_dummies(data)

# Check dataframe data types
data.dtypes

Month                     int64
Day                       int64
Year                      int64
TI                        int64
ISO.tot.dmd             float64
LMP.DA.NH               float64
LMP.RT.NH               float64
UES.Cap                 float64
UES.Sea                 float64
UES.Sea.7.DAYS.AHEAD    float64
NH.RT.MWh               float64
ME.RT.MWh               float64
VT.RT.MWh               float64
CT.RT.MWh               float64
RI.RT.MWh               float64
MA.SE.RT.MWh            float64
MA.WC.RT.MWh            float64
MA.NE.RT.MWh            float64
Wk.Day_Fri                uint8
Wk.Day_Mon                uint8
Wk.Day_Sat                uint8
Wk.Day_Sun                uint8
Wk.Day_Thu                uint8
Wk.Day_Tue                uint8
Wk.Day_Wed                uint8
dtype: object

In [62]:
# Split data into target variable and predictor variables

# Target is the value we want to predict (in this case, MW of UES-Seacoast Load Region)
target = np.array(data['UES.Sea.7.DAYS.AHEAD'])

# Remove the target from the predictors
# axis 1 refers to the columns
predictors = data.drop('UES.Sea.7.DAYS.AHEAD', axis = 1)

# Saving predictor names for later use
predictor_list = list(predictors.columns)

# Convert to numpy array
predictors = np.array(predictors)

In [63]:
# Split the data into training and testing sets using sklearn
train_predictors, test_predictors, train_target, test_target = train_test_split(predictors, target, test_size = 0.25, random_state = 77)


In [64]:
# Check shape of train and test data
print('Training Predictors Shape:', train_predictors.shape)
print('Training Target Shape:', train_target.shape)
print('Testing Predictors Shape:', test_predictors.shape)
print('Testing Target Shape:', test_target.shape)

Training Predictors Shape: (59040, 24)
Training Target Shape: (59040,)
Testing Predictors Shape: (19680, 24)
Testing Target Shape: (19680,)


In [65]:
# Instantiate model with 300 decision trees
start_time = time.time()

rf = RandomForestRegressor(n_estimators = 300, random_state = 77)
# Train the model on training data
rf.fit(train_predictors, train_target)

end_time = time.time()

print("--- %s seconds ---" % (end_time - start_time))

--- 368.5899999141693 seconds ---


In [66]:
# Use the forest's predict method on the test data
predictions = rf.predict(test_predictors)

# Calculate max and mean test_target value
test_target_max_7 = max(test_target)
test_target_mean_7 = np.mean(test_target)

# Calculate the absolute errors, calculate max and mean absolute error
errors = abs(predictions - test_target)
max_absolute_error_7 = max(errors)
mean_absolute_error_7 = np.mean(errors)

# Calculate mean absolute percentage error (MAPE), and Accuracy
mape_7 = 100 * (errors / test_target)
accuracy_7 = 100 - np.mean(mape_7)

# Print out some statistics, including accuracy
print('Target Variable Max Value:', round(test_target_max_7, 2), 'MW')
print('Max Absolute Error:', round(max_absolute_error_7, 2), 'MW')
print('Mean Target Variable Value:', round(test_target_mean_7, 2), 'MW')
print('Mean Absolute Error:', round(mean_absolute_error_7, 2), 'MW')
print('Accuracy:', round(accuracy_7, 2), '%.')

Target Variable Max Value: 168.42 MW
Max Absolute Error: 56.66 MW
Mean Target Variable Value: 77.03 MW
Mean Absolute Error: 4.06 MW
Accuracy: 94.75 %.


In [67]:
# Get numerical feature importances
importances = list(rf.feature_importances_)

# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(predictor_list, importances)]

# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)

# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

Variable: UES.Sea              Importance: 0.65
Variable: Month                Importance: 0.05
Variable: MA.SE.RT.MWh         Importance: 0.05
Variable: TI                   Importance: 0.04
Variable: Day                  Importance: 0.03
Variable: Year                 Importance: 0.02
Variable: LMP.DA.NH            Importance: 0.02
Variable: LMP.RT.NH            Importance: 0.02
Variable: UES.Cap              Importance: 0.02
Variable: ME.RT.MWh            Importance: 0.02
Variable: VT.RT.MWh            Importance: 0.02
Variable: ISO.tot.dmd          Importance: 0.01
Variable: NH.RT.MWh            Importance: 0.01
Variable: CT.RT.MWh            Importance: 0.01
Variable: RI.RT.MWh            Importance: 0.01
Variable: MA.WC.RT.MWh         Importance: 0.01
Variable: MA.NE.RT.MWh         Importance: 0.01
Variable: Wk.Day_Fri           Importance: 0.0
Variable: Wk.Day_Mon           Importance: 0.0
Variable: Wk.Day_Sat           Importance: 0.0
Variable: Wk.Day_Sun           Importance: 

##### Compare Performance of the 7 Random Forest Models

In [68]:
### Print out performance metrics from the seven Random Forest Models

print('R.F. Model 1 - Predict UES Load 1 Day Ahead')
print('Target Variable Max Value:', round(test_target_max_1, 2), 'MW')
print('Max Absolute Error:', round(max_absolute_error_1, 2), 'MW')
print('Mean Target Variable Value:', round(test_target_mean_1, 2), 'MW')
print('Mean Absolute Error:', round(mean_absolute_error_1, 2), 'MW')
print('Accuracy:', round(accuracy_1, 2), '%.')
print('')

print('R.F. Model 2 - Predict UES Load 2 Days Ahead')
print('Target Variable Max Value:', round(test_target_max_2, 2), 'MW')
print('Max Absolute Error:', round(max_absolute_error_2, 2), 'MW')
print('Mean Target Variable Value:', round(test_target_mean_2, 2), 'MW')
print('Mean Absolute Error:', round(mean_absolute_error_2, 2), 'MW')
print('Accuracy:', round(accuracy_2, 2), '%.')
print('')

print('R.F. Model 3 - Predict UES Load 3 Days Ahead')
print('Target Variable Max Value:', round(test_target_max_3, 2), 'MW')
print('Max Absolute Error:', round(max_absolute_error_3, 2), 'MW')
print('Mean Target Variable Value:', round(test_target_mean_3, 2), 'MW')
print('Mean Absolute Error:', round(mean_absolute_error_3, 2), 'MW')
print('Accuracy:', round(accuracy_3, 2), '%.')
print('')

print('R.F. Model 4 - Predict UES Load 4 Days Ahead')
print('Target Variable Max Value:', round(test_target_max_4, 2), 'MW')
print('Max Absolute Error:', round(max_absolute_error_4, 2), 'MW')
print('Mean Target Variable Value:', round(test_target_mean_4, 2), 'MW')
print('Mean Absolute Error:', round(mean_absolute_error_4, 2), 'MW')
print('Accuracy:', round(accuracy_4, 2), '%.')
print('')

print('R.F. Model 5 - Predict UES Load 5 Days Ahead')
print('Target Variable Max Value:', round(test_target_max_5, 2), 'MW')
print('Max Absolute Error:', round(max_absolute_error_5, 2), 'MW')
print('Mean Target Variable Value:', round(test_target_mean_5, 2), 'MW')
print('Mean Absolute Error:', round(mean_absolute_error_5, 2), 'MW')
print('Accuracy:', round(accuracy_5, 2), '%.')
print('')

print('R.F. Model 6 - Predict UES Load 6 Days Ahead')
print('Target Variable Max Value:', round(test_target_max_6, 2), 'MW')
print('Max Absolute Error:', round(max_absolute_error_6, 2), 'MW')
print('Mean Target Variable Value:', round(test_target_mean_6, 2), 'MW')
print('Mean Absolute Error:', round(mean_absolute_error_6, 2), 'MW')
print('Accuracy:', round(accuracy_6, 2), '%.')
print('')

print('R.F. Model 7 - Predict UES Load 7 Days Ahead')
print('Target Variable Max Value:', round(test_target_max_7, 2), 'MW')
print('Max Absolute Error:', round(max_absolute_error_7, 2), 'MW')
print('Mean Target Variable Value:', round(test_target_mean_7, 2), 'MW')
print('Mean Absolute Error:', round(mean_absolute_error_7, 2), 'MW')
print('Accuracy:', round(accuracy_7, 2), '%.')
print('')

R.F. Model 1 - Predict UES Load 1 Day Ahead
Target Variable Max Value: 166.65 MW
Max Absolute Error: 51.4 MW
Mean Target Variable Value: 77.05 MW
Mean Absolute Error: 3.5 MW
Accuracy: 95.58 %.

R.F. Model 2 - Predict UES Load 2 Days Ahead
Target Variable Max Value: 168.43 MW
Max Absolute Error: 53.69 MW
Mean Target Variable Value: 76.92 MW
Mean Absolute Error: 4.21 MW
Accuracy: 94.64 %.

R.F. Model 3 - Predict UES Load 3 Days Ahead
Target Variable Max Value: 168.42 MW
Max Absolute Error: 54.49 MW
Mean Target Variable Value: 77.07 MW
Mean Absolute Error: 4.12 MW
Accuracy: 94.66 %.

R.F. Model 4 - Predict UES Load 4 Days Ahead
Target Variable Max Value: 168.43 MW
Max Absolute Error: 54.32 MW
Mean Target Variable Value: 77.13 MW
Mean Absolute Error: 3.97 MW
Accuracy: 94.86 %.

R.F. Model 5 - Predict UES Load 5 Days Ahead
Target Variable Max Value: 165.69 MW
Max Absolute Error: 59.83 MW
Mean Target Variable Value: 77.08 MW
Mean Absolute Error: 3.93 MW
Accuracy: 94.89 %.

R.F. Model 6 - Pre

In [69]:
# Create a dataframe of performance metrics of 7 models
model_number = [1,2,3,4,5,6,7]

test_target_max_values = [test_target_max_1, test_target_max_2, test_target_max_3, test_target_max_4, 
                          test_target_max_6, test_target_max_6, test_target_max_7]

max_absolute_error_values =[max_absolute_error_1, max_absolute_error_2, max_absolute_error_3, max_absolute_error_4, 
                            max_absolute_error_5, max_absolute_error_6, max_absolute_error_7]

test_target_mean_values = [test_target_mean_1, test_target_mean_2, test_target_mean_3, test_target_mean_4, 
                           test_target_mean_5, test_target_mean_6, test_target_mean_7]

mean_absolute_error_values = [mean_absolute_error_1, mean_absolute_error_2, mean_absolute_error_3, 
                              mean_absolute_error_4, mean_absolute_error_5, mean_absolute_error_6, 
                              mean_absolute_error_7]

accuracy_values = [accuracy_1, accuracy_2, accuracy_3, accuracy_4, accuracy_5, accuracy_6, accuracy_7]

### Create pandas dataframe using metrics
metrics_df = {'Model Number': model_number,
              'Test Target Max': test_target_max_values, 
              'Max Absolute Error': max_absolute_error_values,
              'Test Target Mean': test_target_mean_values,
              'Mean Absolute Error': mean_absolute_error_values,
              'Accuracy': accuracy_values}

metrics_df = pd.DataFrame(metrics_df)
metrics_df


,Model Number,Test Target Max,Max Absolute Error,Test Target Mean,Mean Absolute Error,Accuracy
0,1,166.649,51.400550,77.047343,3.504857,95.582884
1,2,168.430,53.685997,76.918785,4.214462,94.642541
2,3,168.420,54.490740,77.070027,4.115983,94.661963
3,4,168.430,54.322610,77.126991,3.974207,94.856136
4,5,166.733,59.828940,77.079134,3.932354,94.890631
5,6,166.733,56.439853,76.964268,4.031080,94.726121
6,7,168.420,56.656283,77.033948,4.061459,94.747932


In [70]:
# Write out a .csv of the performance metrics
# metrics_df.to_csv('metrics_RandomForest.csv', index=False)